In [ ]:
import os
import time
import pickle

import cv2
from shapely.geometry import Polygon
import numpy.typing as npt

In [ ]:
from typing import List, Tuple

In [ ]:
from optimized_ingestion.camera_config import camera_config, CameraConfig
from optimized_ingestion.video import Video

In [ ]:
from optimized_ingestion.detection_estimation.segment_mapping import map_imgsegment_roadsegment, CameraSegmentMapping
from optimized_ingestion.detection_estimation.utils import trajectory_3d
from optimized_ingestion.detection_estimation.detection_estimation import construct_all_detection_info, detection_to_img_segment, obj_detection, generate_sample_plan, DetectionInfo, samplePlan

In [ ]:
from optimized_ingestion.pipeline import Pipeline
from optimized_ingestion.payload import Payload

from optimized_ingestion.stages.decode_frame.parallel_decode_frame import ParallelDecodeFrame
from optimized_ingestion.stages.detection_2d.yolo_detection import YoloDetection

In [ ]:
BOSTON_VIDEO_DIR = os.path.join(os.environ['NUSCENES_PROCESSED_DATA'], 'videos/boston-seaport')
BOSTON_VIDEO_DIR

In [ ]:
car_loc3d_ground_truth = [(1991, 874), (1949.181, 873.164)]

In [ ]:
with open(os.path.join(BOSTON_VIDEO_DIR, 'frames.pickle'), 'rb') as f:
    videoconfigs = pickle.load(f)

In [ ]:
def display_detection(test_file_path: str, full_img_detection):
    test_frame = cv2.imread(test_file_path)
    for obj_idx, detection in full_img_detection.items():
        obj_cls, bbox = detection
        if obj_cls == 'car':
            x,y,w,h = list(map(int,bbox))
            cv2.rectangle(test_frame,(x-w//2,y-h//2),(x+w//2,y+h//2),(0,255,0),2)
            cv2.putText(test_frame, '_'.join([obj_cls, str(obj_idx)]), (x+w//2+5,y+h//2+5),0,0.3,(0,255,0))
    cv2.imshow('detection', test_frame)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [ ]:
def get_video(videoname: str, framesdict: dict) -> "Video":
    videodata = framesdict[videoname]
    filename = videodata['filename']
    frames = videodata['frames']
    configs = [camera_config(*f, 0) for f in frames]
    return Video(os.path.join(BOSTON_VIDEO_DIR, filename), configs, videodata['start'])

In [ ]:
# ego car trajectory
def prepare_ego(test_video: str) -> "Tuple[Video, List[trajectory_3d]]":
    video = get_video(test_video, videoconfigs)
    ego_trajectory = [trajectory_3d(f.ego_translation, f.timestamp) for f in video]
    return video, ego_trajectory

In [ ]:
def generate_sample_plan_once(
    video: "str",
    ego_config: "CameraConfig",
    mapping: "List[CameraSegmentMapping]",
    next_frame_num: "int",
    car_loc3d=None,
    target_car_detection=None,
    all_detection_info: "List[obj_detection]" = None
) -> "Tuple[samplePlan, None]":
    # if all_detection_info is None:
    #     assert target_car_detection and car_loc3d
    #     x,y,w,h = list(map(int, target_car_detection))
    #     car_loc2d = (x, y+h//2)
    #     car_bbox2d = (x-w//2,y-h//2,x+w//2,y+h//2)
    #     car_bbox3d = None
    #     all_detections = []
    #     all_detections.append(obj_detection('car_1', car_loc3d, car_loc2d, car_bbox3d, car_bbox2d))
    #     all_detection_info = construct_all_detection_info(cam_segment_mapping, ego_trajectory, ego_config, all_detections)
    if all_detection_info:
        print(all_detection_info[0].road_type)
    next_sample_plan = generate_sample_plan(video, next_frame_num, all_detection_info,  50)
    # next_frame = None
    next_sample_frame_info = next_sample_plan.get_next_sample_frame_info()
    if next_sample_frame_info:
        next_sample_frame_name, next_sample_frame_num, _ = next_sample_frame_info
        print("next frame name", next_sample_frame_name)
        print("next frame num", next_sample_frame_num)
    #     print(next_sample_plan.action)
        # TODO: should not read next frame -> get the next frame from frames.pickle
        # next_frame = cv2.imread(test_img_base_dir+next_sample_frame_name)
#         cv2.imshow("next_frame", next_frame)
#         cv2.waitKey(0)
#         cv2.destroyAllWindows()
    return next_sample_plan, None

In [ ]:
def construct_estimated_all_detection_info(
    detections: "npt.NDArray",
    cam_segment_mapping: "List[CameraSegmentMapping]",
    ego_config: "CameraConfig",
    ego_trajectory: "trajectory_3d"
) -> "List[DetectionInfo]":
    all_detections = []
    for det in detections:
        bbox = det[:4]
        obj_cls = det[5]
        x, y, x2, y2 = list(map(int,bbox))
        w = x2 - x
        h = y2 - y
        car_loc2d = (x + w // 2, y+h//2)
#         print(car_loc2d)
        car_bbox2d = ((x-w//2, y-h//2), (x+w//2, y+h//2))
        car_bbox3d = None
        estimate_3d = detection_to_img_segment(car_loc2d, cam_segment_mapping)
        if estimate_3d and estimate_3d.road_segment_info.segment_type in ['lane', 'laneSection']:
            car_loc3d = tuple(Polygon(estimate_3d.road_segment_info.segment_polygon).centroid.coords)
#             print(tuple(car_loc3d))
            all_detections.append(obj_detection('car_1', car_loc3d, car_loc2d, car_bbox3d, car_bbox2d))
    print("all_detections", all_detections)
    all_detection_info = construct_all_detection_info(cam_segment_mapping, ego_config, ego_trajectory, all_detections)
    return all_detection_info

In [ ]:
def dry_run(
    payload: "Payload",
    start_frame_num: "int",
    ego_trajectory: "List[trajectory_3d]",
    video: "str"
):
    skipped_frame_num = []
    next_frame_num = start_frame_num
    action_type_counts = {}
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    display_video = cv2.VideoWriter(f'sampled_frames_{video.replace("/", "_")}.avi',fourcc, 10, (1600, 900))
    start_time = time.time()
    total_detection_time = 0
    total_sample_plan_time = 0
    for i in range(len(payload.video)-1):
        current_ego_config = payload.video[i]
        if i != next_frame_num:
            skipped_frame_num.append(i)
            continue
        next_frame_num = i + 1
        cam_segment_mapping = map_imgsegment_roadsegment(current_ego_config)
        print("mapping length", len(cam_segment_mapping))
        # current_frame = test_img_base_dir + current_ego_config['fileName']
        # display_video.write(cv2.imread(current_frame))
        start_detection_time = time.time()
        all_detection_info = construct_estimated_all_detection_info(YoloDetection.get(payload)[i][0], cam_segment_mapping, current_ego_config, ego_trajectory)
        total_detection_time += time.time()-start_detection_time
        start_generate_sample_plan = time.time()
        next_sample_plan, _ = generate_sample_plan_once(payload.video, current_ego_config, cam_segment_mapping, next_frame_num, all_detection_info=all_detection_info)
        total_sample_plan_time += time.time() - start_generate_sample_plan
        next_action_type = next_sample_plan.get_action_type()
        if next_action_type not in action_type_counts:
            action_type_counts[next_action_type] = 1
        else:
            action_type_counts[next_action_type] += 1
        next_frame_num = next_sample_plan.get_next_frame_num(next_frame_num)

    display_video.release()
    print("sorted_ego_config_length", len(payload.video))
    print("number of skipped", len(skipped_frame_num))
    print(skipped_frame_num)
    print(action_type_counts)
    total_run_time = time.time()-start_time
    num_runs = len(payload.video) - len(skipped_frame_num)
    print("total_run_time", total_run_time)
    print("avg run time", total_run_time/num_runs)
    print("total_detection_time", total_detection_time)
    print("avg detection time", total_detection_time/num_runs)
    print("total_generate_sample_plan_time", total_sample_plan_time)
    print("avg generate_sample_plan time", total_sample_plan_time/num_runs)

In [ ]:
# Construct pipeline -> decoding frames -> 2D Detection -> 3D Detection
pipeline = Pipeline()
pipeline.add_filter(ParallelDecodeFrame())
pipeline.add_filter(YoloDetection())

test_video1 = 'scene-0757-CAM_FRONT'
video1, ego_trajectory1 = prepare_ego(test_video1)
payload1 = pipeline.run(Payload(video1))

In [ ]:
dry_run(payload1, 0, ego_trajectory1, test_video1)